## LAB-24 Open Data 第三方支付服務業

* 資料來源：政府開放資料平台<br>
  [公司登記(依營業項目別)－第三方支付服務業](https://data.gov.tw/dataset/22184)<br>
  [商業登記(依營業項目別)－第三方支付服務業](https://data.gov.tw/dataset/108381)<br>
* 資料下載網址：<br>
  公司登記 https://data.gcis.nat.gov.tw/od/file?oid=993D3D8B-3993-4279-9A09-0746EB35062B <br>
  商業登記 https://data.gcis.nat.gov.tw/od/file?oid=702A3D64-EC36-49E4-8605-738033B8B231 <br>
* 「公司登記」與「商業登記」的區別 -> [Gemini](https://g.co/gemini/share/783116509616)

In [1]:
# 檢查套件是否已安裝
!conda list "pandas|numpy|openpyxl|requests|tqdm"

# packages in environment at /opt/homebrew/Caskroom/miniforge/base/envs/lab:
#
# Name                    Version                   Build  Channel
numpy                     2.2.4           py313h41a2e72_0    conda-forge
openpyxl                  3.1.5           py313h90caf49_1    conda-forge
pandas                    2.2.3           py313h47b39a6_1    conda-forge
requests                  2.32.3             pyhd8ed1ab_1    conda-forge
tqdm                      4.67.1             pyhd8ed1ab_1    conda-forge


In [2]:
import pandas as pd
import numpy as np
import requests 
from tqdm.notebook import tqdm 
import csv
#from rich import print 

In [3]:
def download_data(url, fname) -> bool:
    """
    參數
        url    網址
        fname  儲存檔名

    return
        bool   成功或失敗
    """

    try:
        # 發送 GET 請求，設定 stream=True 以啟用串流下載
        response = requests.get(url, stream=True)
    
        # 確認請求成功
        response.raise_for_status()
    
        # 取得檔案大小（單位：位元組）
        total_size = int(response.headers.get('content-length', 0))
        
    except requests.exceptions.HTTPError as err:
        print(f'HTTP 錯誤：{err}')
        return False
        
    except requests.exceptions.RequestException as err:
        print(f'HTTP 錯誤：{err}')
        return False 
        
    # 每次讀取德區塊大小
    block_size = 1024
    
    # for debug
    read_bytes = 0

    bar = tqdm(desc=fname, total=total_size, unit='B', unit_scale=True, unit_divisor=1024)
    
    bar.refresh()

    # 準備寫入檔案
    with open(fname, 'wb') as file:
        try:    
            # 逐塊下載資料
            for chunk in response.iter_content(chunk_size=block_size):
                if chunk:  # 確保不是空值
                    read_bytes += len(chunk)
                    #print(f"{read_bytes}/{total_size} = {read_bytes/total_size:.2%}")
    
                    bar.update(len(chunk))
                    
                    # 將下載的資料寫入檔案並更新進度條
                    file.write(chunk)
                else:
                    # End of Stream
                    pass
        except requests.exceptions.RequestException as err:
            print(f'HTTP REQUEST 錯誤：{err}')
            return False 

    bar.refresh()
    #bar.close()            
    #del bar 
    
    return True


In [4]:
url1 = r"https://data.gcis.nat.gov.tw/od/file?oid=993D3D8B-3993-4279-9A09-0746EB35062B"
csv1 = "第三方支付-公司登記.csv"

#download_data(url1, csv1)


In [5]:
url2 = r"https://data.gcis.nat.gov.tw/od/file?oid=702A3D64-EC36-49E4-8605-738033B8B231"
csv2 = "第三方支付-營業登記.csv"

#download_data(url2, csv2)

In [6]:
# 讀取 CSV 檔
df1 = pd.read_csv(csv1)

df1.head()

,統一編號,公司名稱,負責人,公司地址,資本總額,實收資本額,在境內營運資金,公司狀態,產製日期
0,1200,形韻科技股份有限公司,林O民,新竹市東區中華路二段198號1樓及地下一樓,1000000,1000000,,核准設立,20250301034640
1,2799,鑫晶生物科技有限公司,陳O英,臺北市大同區市民大道1段209號10樓,1000000,,,核准設立,20250301034640
2,4810,道原智慧服務有限公司,廖O奇,新北市五股區中興路1段162號（2樓）,2000000,,,核准設立,20250301034640
3,4945,顯樂文化事業有限公司,黃O宇,花蓮縣新城鄉新秀村光復路497號2樓之1,100000,,,核准設立,20250301034640
4,5938,瑞盛人力資源管理顧問有限公司,陳O瑞,臺北市信義區信義路4段415號5樓之4,100000,,,核准設立,20250301034640


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16045 entries, 0 to 16044
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   統一編號     16045 non-null  int64 
 1   公司名稱     16045 non-null  object
 2   負責人      11172 non-null  object
 3   公司地址     16045 non-null  object
 4   資本總額     16045 non-null  object
 5   實收資本額    16045 non-null  object
 6   在境內營運資金  16045 non-null  object
 7   公司狀態     16045 non-null  object
 8   產製日期     16045 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1.1+ MB


In [8]:
# 欄位格式有問題：
# 統一編號 -> str
# 資本總額 -> int 
# 實收資本額 -> int
# 在境內營運資金 -> int 
# 產製日期 -> data 

In [9]:
# 更換欄位名稱
new_col = {"統一編號":"IDNO",
           "公司名稱":"NAME",
           "負責人":"OWNER",
           "公司地址":"ADDRESS",
           "資本總額":"CAPITAL",
           "實收資本額":"CAPITAL_NET",
           "在境內營運資金":"CAPITAL_LOCAL",
           "公司狀態":"STATUS",
           "產製日期":"DATA_DATE",
          }

df1 = df1.rename(columns=new_col)
df1.columns

Index(['IDNO', 'NAME', 'OWNER', 'ADDRESS', 'CAPITAL', 'CAPITAL_NET',
       'CAPITAL_LOCAL', 'STATUS', 'DATA_DATE'],
      dtype='object')

In [10]:
# 資料日期欄為 int64 轉換為日期
df1['DATA_DATE'] = pd.to_datetime(df1['DATA_DATE'].astype(str), format='%Y%m%d%H%M%S').dt.date
df1.head(5).T

,0,1,2,3,4
IDNO,1200,2799,4810,4945,5938
NAME,形韻科技股份有限公司,鑫晶生物科技有限公司,道原智慧服務有限公司,顯樂文化事業有限公司,瑞盛人力資源管理顧問有限公司
OWNER,林O民,陳O英,廖O奇,黃O宇,陳O瑞
ADDRESS,新竹市東區中華路二段198號1樓及地下一樓,臺北市大同區市民大道1段209號10樓,新北市五股區中興路1段162號（2樓）,花蓮縣新城鄉新秀村光復路497號2樓之1,臺北市信義區信義路4段415號5樓之4
CAPITAL,1000000,1000000,2000000,100000,100000
CAPITAL_NET,1000000,,,,
CAPITAL_LOCAL,,,,,
STATUS,核准設立,核准設立,核准設立,核准設立,核准設立
DATA_DATE,2025-03-01,2025-03-01,2025-03-01,2025-03-01,2025-03-01


In [11]:
cap = pd.to_numeric(df1["CAPITAL"].str.strip()).fillna(0).astype(int)
df1["CAPITAL"] = cap

In [12]:
cap = pd.to_numeric(df1["CAPITAL_NET"].str.strip()).fillna(0).astype(int)
df1["CAPITAL_NET"] = cap

In [13]:
cap = pd.to_numeric(df1["CAPITAL_LOCAL"].str.strip()).fillna(0).astype(int)
df1["CAPITAL_LOCAL"] = cap

In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16045 entries, 0 to 16044
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   IDNO           16045 non-null  int64 
 1   NAME           16045 non-null  object
 2   OWNER          11172 non-null  object
 3   ADDRESS        16045 non-null  object
 4   CAPITAL        16045 non-null  int64 
 5   CAPITAL_NET    16045 non-null  int64 
 6   CAPITAL_LOCAL  16045 non-null  int64 
 7   STATUS         16045 non-null  object
 8   DATA_DATE      16045 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.1+ MB


In [15]:
# 讀取 CSV 檔
df2 = pd.read_csv(csv2)

df2.head()

,統一編號,商業名稱,商業地址,登記狀態
0,261367,禮享家企業社,臺南市安平區建平里中華西路2段317號6樓,核准設立
1,267117,福鑫工程實業社,臺南市安平區建平里建平九街60巷13弄6號1樓,核准設立
2,277430,氶勳企業社,臺南市東區泉南里東門路1段84之1號7樓,停業
3,286965,龍燕車業,臺南市東區成大里前鋒路128號,核准設立
4,288942,富士辦公資訊設備,臺南市安南區南興里安中路5段442號,核准設立


In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237 entries, 0 to 2236
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   統一編號    2237 non-null   int64 
 1   商業名稱    2237 non-null   object
 2   商業地址    2237 non-null   object
 3   登記狀態    2237 non-null   object
dtypes: int64(1), object(3)
memory usage: 70.0+ KB


In [17]:
# 更換欄位名稱
new_col = {"統一編號":"IDNO",
           "商業名稱":"NAME",
           "商業地址":"ADDRESS",
           "登記狀態":"STATUS",
          }

df2 = df2.rename(columns=new_col)
df2.columns

Index(['IDNO', 'NAME', 'ADDRESS', 'STATUS'], dtype='object')

In [18]:
# 檢查 info() 的輸出
# 「統一編號」之型別為 int64
# 進行資料清理及轉換

## 資料清理 
1. 統編須轉換為字串左補零

### 統編左補零
> AI：如何將 DataFrame 的數值欄位轉換為左補零的固定長度字串

In [19]:
df1["IDNO"] = df1["IDNO"].astype(str).str.zfill(8)


In [20]:
df2["IDNO"] = df2["IDNO"].astype(str).str.zfill(8)


In [21]:
df1["IDNO"].head(10)

0    00001200
1    00002799
2    00004810
3    00004945
4    00005938
5    00006438
6    00008610
7    00008843
8    00013634
9    00015823
Name: IDNO, dtype: object

In [22]:
df2["IDNO"].head(10)

0    00261367
1    00267117
2    00277430
3    00286965
4    00288942
5    00289148
6    00306630
7    00539390
8    00541874
9    00815769
Name: IDNO, dtype: object

### 檢查「公司登記」與「商業登記」的資料關係

In [23]:
#使用集合運算
set_idno1 = set(df1["IDNO"].unique())
set_idno2 = set(df2["IDNO"].unique())

In [24]:
print(f"交集筆數：{len(set_idno1 & set_idno2):,d}")

交集筆數：0


In [25]:
# 二者無交集，無合併(JOIN)意義

### 新增「縣市」欄位

In [26]:
# 新增計算欄位 CITY = 地址前三個字
df1["CITY"] = df1["ADDRESS"].str.slice(0,3)
df1["CITY"].value_counts(dropna=False)

CITY
臺北市    5458
新北市    2842
臺中市    2601
高雄市    1752
桃園市    1156
臺南市     572
彰化縣     224
新竹縣     216
新竹市     211
屏東縣     143
基隆市     118
嘉義市     105
苗栗縣     103
花蓮縣     100
南投縣      96
宜蘭縣      95
雲林縣      92
嘉義縣      54
臺東縣      52
金門縣      34
澎湖縣      11
新竹科       5
連江縣       5
Name: count, dtype: int64

In [27]:
# 新增計算欄位 CITY = 地址前三個字
df2["CITY"] = df2["ADDRESS"].str.slice(0,3)
df2["CITY"].value_counts(dropna=False)

CITY
新北市    484
臺中市    385
高雄市    358
臺南市    212
臺北市     94
彰化縣     93
花蓮縣     74
新竹市     59
屏東縣     59
苗栗縣     52
雲林縣     50
宜蘭縣     46
臺東縣     42
南投縣     41
新竹縣     40
嘉義市     39
嘉義縣     38
基隆市     35
桃園市     23
金門縣      8
連江縣      3
澎湖縣      2
Name: count, dtype: int64

### 檢查重複

In [28]:
df1[df1["IDNO"].duplicated()].sort_values(by="IDNO")


,IDNO,NAME,OWNER,ADDRESS,CAPITAL,CAPITAL_NET,CAPITAL_LOCAL,STATUS,DATA_DATE,CITY


In [29]:
# 查無重複 IDNO 
# 「公司登記」為 unique 

In [30]:
# 檢查 商業登記 的 IDNO 
df2[df2["IDNO"].duplicated(keep=False)].sort_values(by=["IDNO"])

,IDNO,NAME,ADDRESS,STATUS,CITY
13,01547669,個人種子資訊工作室,花蓮縣吉安鄉慶豐村慶北一街260之3號4樓,停業,花蓮縣
14,01547669,個人種子資訊工作室,桃園市龍潭區三和里渴望一路280巷8號4樓,遷他縣市,桃園市
16,02139212,傑狗影像工作室,新北市蘆洲區長安街285巷16號4樓,遷他縣市,新北市
17,02139212,傑狗影像工作室,臺南市中西區開山里開山路245號4樓之1,核准設立,臺南市
22,02438400,鈞鉉企業社,新北市板橋區民生路1段1號10樓之2,遷他縣市,新北市
...,...,...,...,...,...
2226,95998784,球點支付數位科技商行,高雄市三民區清華街２３６號２樓,停業,高雄市
2228,98961868,烜統運動用品社,彰化縣員林市光明里光明街３１０號１樓,遷他縣市,彰化縣
2229,98961868,烜統運動用品社,臺中市西區忠誠里臺灣大道二段５０１號１８樓之１,歇業／撤銷,臺中市
2231,99345738,維果製作工作室,新北市新店區十四張路３８３號１２樓之２,核准設立,新北市


In [31]:
# 商業登記 共 216筆 IDNO 重複

In [32]:
# 計算重複次數
df2_dedup = df2.groupby(['IDNO']).size().reset_index(name="CNT")

df2_dedup["CNT"].value_counts(dropna=False)


CNT
1    2021
2     103
3       2
4       1
Name: count, dtype: int64

In [33]:
# 篩選重複次數大於 2 的資料列
df3 = df2_dedup[df2_dedup['CNT'] > 2]

df3

,IDNO,CNT
102,21579891,4
742,76477594,3
1166,85387525,3


In [34]:
# 總共 3個統編：21579891, 76477594, 85387525
# 檢視 3個統編的詳細資料
df2[df2['IDNO'].isin(df3['IDNO'].to_list())].sort_values('IDNO')

,IDNO,NAME,ADDRESS,STATUS,CITY
113,21579891,民營政福興業社,宜蘭縣三星鄉天山村三星路八段７號1樓,核准設立,宜蘭縣
114,21579891,民營政福興業社,新北市中和區中山路２段４０５號３樓,遷他縣市,新北市
115,21579891,民營政福興業社,臺中市西屯區何安里臺灣大道二段888號9樓之18,核准設立,臺中市
116,21579891,民營政福興業社,臺東縣太麻里鄉美和村荒野４８之１１號,核准設立,臺東縣
795,76477594,久利笙企業社,桃園市桃園區中寧里中正路927號十樓,核准設立,桃園市
796,76477594,久利笙企業社,新北市新店區北新路1段257號,核准設立,新北市
797,76477594,久利笙企業社,臺北市士林區華齡街1巷8號1樓,核准設立,臺北市
1234,85387525,福金士商行,桃園市觀音區三和里文化路坑尾段360巷31弄3號2樓,遷他縣市,桃園市
1235,85387525,福金士商行,新北市板橋區華江一路576號13樓,廢止,新北市
1236,85387525,福金士商行,臺北市大安區忠孝東路4段147巷1號7樓之1,遷他縣市,臺北市


In [35]:
# 21579891 有 3筆 「核准設立」
# 76477594 3筆都是「核准設立」，均位於不同縣市
# 85387525 沒有1筆「核准設立」

### 分析「狀態」的欄位值

#### 公司登記 的「狀態」

In [36]:
df1['STATUS'].value_counts()

STATUS
核准設立           10954
解散              4092
廢止               763
核准登記             100
廢止登記              53
核准設立，但已命令解散       48
撤回認許              17
撤回登記               8
廢止認許               3
合併解散               2
撤銷                 2
破產                 2
撤銷公司設立             1
Name: count, dtype: int64

In [37]:
# 使用 map() 函數轉換，並處理遺漏值
mapping = {"核准設立":"設立",
           "解散":"解散",
           "廢止":"廢止",
           "核准登記":"登記",
           "廢止登記":"廢止",
           "核准設立，但已命令解散":"解散",
           "撤回認許":"廢止",
           "撤回登記":"廢止",
           "廢止認許":"廢止",
           "合併解散":"解散",
           "撤銷":"廢止",
           "破產":"解散",
           "撤銷公司設立":"解散",
          }
           
df1['STATUS2'] = df1['STATUS'].map(mapping).fillna('其他')
df1['STATUS2'].value_counts() 

STATUS2
設立    10954
解散     4145
廢止      846
登記      100
Name: count, dtype: int64

#### 商業登記 的 「狀態」

In [38]:
# 商業登記
# 原假設 STATUS 應包含公司申請歷程，如：「核准設立」才會有「停業」或「廢止」
# 但檢視前述IDNO重複資料，並非如此

In [39]:
# 新增(合併)計算欄位 CNT = 重複筆數

df3 = pd.merge(df2, df2_dedup, on='IDNO', how='left')
df3.head()

,IDNO,NAME,ADDRESS,STATUS,CITY,CNT
0,00261367,禮享家企業社,臺南市安平區建平里中華西路2段317號6樓,核准設立,臺南市,1
1,00267117,福鑫工程實業社,臺南市安平區建平里建平九街60巷13弄6號1樓,核准設立,臺南市,1
2,00277430,氶勳企業社,臺南市東區泉南里東門路1段84之1號7樓,停業,臺南市,1
3,00286965,龍燕車業,臺南市東區成大里前鋒路128號,核准設立,臺南市,1
4,00288942,富士辦公資訊設備,臺南市安南區南興里安中路5段442號,核准設立,臺南市,1


In [40]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237 entries, 0 to 2236
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   IDNO     2237 non-null   object
 1   NAME     2237 non-null   object
 2   ADDRESS  2237 non-null   object
 3   STATUS   2237 non-null   object
 4   CITY     2237 non-null   object
 5   CNT      2237 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 105.0+ KB


In [41]:

df3.value_counts(subset=['CNT','STATUS'], dropna = False, sort=False)

CNT  STATUS
1    停業         140
     廢止          69
     核准設立      1082
     歇業／撤銷      728
     申覆（辯）期       2
2    停業          17
     廢止           5
     核准設立        96
     歇業／撤銷       25
     遷他縣市        63
3    廢止           1
     核准設立         3
     遷他縣市         2
4    核准設立         3
     遷他縣市         1
Name: count, dtype: int64

In [42]:
# 以 縣市 + 統編 判斷
df2[df2.duplicated(subset=['IDNO','CITY'],keep=False)]

,IDNO,NAME,ADDRESS,STATUS,CITY


In [43]:
df2['STATUS'].value_counts()

STATUS
核准設立      1184
歇業／撤銷      753
停業         157
廢止          75
遷他縣市        66
申覆（辯）期       2
Name: count, dtype: int64

In [44]:
# 使用 map() 函數轉換，並處理遺漏值
mapping = {"核准設立":"設立",
           "歇業／撤銷":"解散",
           "停業":"解散",
           "廢止":"廢止",
           "遷他縣市":"廢止",
           "申覆（辯）期":"申覆",
          }
           
df2['STATUS2'] = df2['STATUS'].map(mapping).fillna('其他')
df2['STATUS2'].value_counts() 

STATUS2
設立    1184
解散     910
廢止     141
申覆       2
Name: count, dtype: int64

In [45]:
df3 = pd.merge(df1['STATUS'].value_counts(dropna=False),
               df2['STATUS'].value_counts(dropna=False), 
               on='STATUS',
               suffixes=('_1', '_2'), 
               how='outer')

df3 = df3.fillna(0).reset_index()

df3.head(df3.shape[0])


,STATUS,count_1,count_2
0,停業,0.0,157.0
1,合併解散,2.0,0.0
2,廢止,763.0,75.0
3,廢止登記,53.0,0.0
4,廢止認許,3.0,0.0
5,撤回登記,8.0,0.0
6,撤回認許,17.0,0.0
7,撤銷,2.0,0.0
8,撤銷公司設立,1.0,0.0
9,核准登記,100.0,0.0


In [46]:
df3 = pd.merge(df1['STATUS2'].value_counts(dropna=False),
               df2['STATUS2'].value_counts(dropna=False), 
               on='STATUS2',
               suffixes=('_1', '_2'), 
               how='outer')

df3 = df3.fillna(0).reset_index()

df3.head(df3.shape[0])


,STATUS2,count_1,count_2
0,廢止,846.0,141.0
1,申覆,0.0,2.0
2,登記,100.0,0.0
3,解散,4145.0,910.0
4,設立,10954.0,1184.0


### 合併資料集

In [47]:
df1['CATEGORY']='公司登記'
df1.head(1)

,IDNO,NAME,OWNER,ADDRESS,CAPITAL,CAPITAL_NET,CAPITAL_LOCAL,STATUS,DATA_DATE,CITY,STATUS2,CATEGORY
0,00001200,形韻科技股份有限公司,林O民,新竹市東區中華路二段198號1樓及地下一樓,1000000,1000000,0,核准設立,2025-03-01,新竹市,設立,公司登記


In [48]:
df2['CATEGORY'] = '營業登記'
df2.head(1)

,IDNO,NAME,ADDRESS,STATUS,CITY,STATUS2,CATEGORY
0,00261367,禮享家企業社,臺南市安平區建平里中華西路2段317號6樓,核准設立,臺南市,設立,營業登記


In [49]:
df = pd.concat([df1,df2],ignore_index=True)

In [50]:
df['CATEGORY'].value_counts()

CATEGORY
公司登記    16045
營業登記     2237
Name: count, dtype: int64

## 儲存資料清理後的結果

In [51]:
# 刪除非必要欄位
df.info()

del df["STATUS"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18282 entries, 0 to 18281
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   IDNO           18282 non-null  object 
 1   NAME           18282 non-null  object 
 2   OWNER          11172 non-null  object 
 3   ADDRESS        18282 non-null  object 
 4   CAPITAL        16045 non-null  float64
 5   CAPITAL_NET    16045 non-null  float64
 6   CAPITAL_LOCAL  16045 non-null  float64
 7   STATUS         18282 non-null  object 
 8   DATA_DATE      16045 non-null  object 
 9   CITY           18282 non-null  object 
 10  STATUS2        18282 non-null  object 
 11  CATEGORY       18282 non-null  object 
dtypes: float64(3), object(9)
memory usage: 1.7+ MB


In [52]:
# 欄位名稱 更名回中文
new_col = {"CITY":"縣市",
           "IDNO":"統一編號",
           "NAME":"公司名稱",
           "OWNER":"負責人",
           "ADDRESS":"公司地址",
           "CAPITAL":"資本總額",
           "CAPITAL_NET":"實收資本額",
           "CAPITAL_LOCAL":"在境內營運資金",
           "STATUS2":"狀態",
           "DATA_DATE":"產製日期",
           "CATEGORY":"類別",
          }
df = df.rename(columns=new_col)
df.columns


Index(['統一編號', '公司名稱', '負責人', '公司地址', '資本總額', '實收資本額', '在境內營運資金', '產製日期', '縣市',
       '狀態', '類別'],
      dtype='object')

In [53]:
# 儲存為 CSV
csvf = "lab-24.csv"
df.to_csv(csvf, index=False, encoding='utf-8-sig',quoting = csv.QUOTE_NONNUMERIC)

In [54]:
# 儲存為 Excel
# 儲存為 Excel
xlsx = "lab-24.xlsx"
with pd.ExcelWriter(xlsx) as writer:
    df.to_excel(writer, sheet_name='第三方支付', index=False)
    